<a href="https://colab.research.google.com/github/gis2010/voice_activity_detection/blob/master/7_Reasoning_Paradigms_in_HDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Accompanying talk](https://youtu.be/gv9IBpooQAg?t=8597)
## [Slide deck](https://docs.google.com/presentation/d/1tJHTOdMAT6S_gQSsfkUMmbN6GTrVq5hCOofnM-cLr-c/edit?usp=sharing)

In [ ]:
!pip install bhv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bhv: filename=bhv-1.4.1-cp310-cp310-linux_x86_64.whl size=629394 sha256=ecf87d34870f6510b25ca20be63b637d98ab52d0808424f9fb8626821faca2b2
  Stored in directory: /root/.cache/pip/wheels/41/35/50/aa34c6bbdfd012395777d94cbcf36fc120f518bd9052a45e46
Successfully built bhv


In [ ]:
from bhv.np import NumPyBoolBHV as BHV, NumPyBoolPermutation as Perm
from random import random
from math import hypot, atan2

## generalization by factoring (XOR factors over MAJORITY)

In [ ]:
Mary, home = BHV.nrand(2)
walked, ran, drove, flew = BHV.nrand(4)

In [ ]:
datapoints = [
    Mary ^ walked ^ home,
    Mary ^ ran ^ home,
    Mary ^ drove ^ home,
]

In [ ]:
transportation = BHV.majority([walked, ran, drove, flew])

expected = Mary ^ transportation ^ home

assert BHV.majority(datapoints).related(expected)
assert BHV.majority(datapoints) == Mary ^ BHV.majority([walked, ran, drove]) ^ home

## reasoning by analogy (XOR can be used transitively)

In [ ]:
name, capital_city, money = BHV.nrand(3)
united_states, washington_dc, dollar = BHV.nrand(3)
mexico, mexico_city, peso = BHV.nrand(3)

USA = BHV.majority([name ^ united_states, capital_city ^ washington_dc, money ^ dollar])
MEX = BHV.majority([name ^ mexico, capital_city ^ mexico_city, money ^ peso])

Pair = USA ^ MEX

In [ ]:
# think about {"name": "united_states"}["name"]

assert (USA ^ name).related(united_states)

In [ ]:
dollar_of_mexico = dollar ^ Pair

assert peso.related(dollar_of_mexico)
assert peso.hamming(dollar_of_mexico) < dollar.hamming(dollar_of_mexico)

In [ ]:
mexico_city_of_usa = mexico_city ^ Pair

assert washington_dc.related(mexico_city_of_usa)
assert washington_dc.hamming(mexico_city_of_usa) < mexico_city.hamming(mexico_city_of_usa)

## transporting by a concept (XOR maintains relative operations)

In [ ]:
focus = BHV.rand()
unifocal = BHV.rand()
bifocal = unifocal.flip_frac(0.2)

shape = BHV.rand()
flat = shape.flip_frac(0.2)
round = shape.flip_frac(0.2)
shape_twisting = shape.flip_frac(.2)

shape, flat, round, point_symmetric = BHV.nrand(4)

circle = BHV.majority([shape, flat, round, point_symmetric, focus ^ unifocal])
oval = BHV.majority([shape, flat, round, focus ^ bifocal])

In [ ]:
def P(circle_, oval_, point_symmetric_, unifocal_, bifocal_):
    assert circle_.related(oval_)
    assert circle_.related(point_symmetric_)
    assert not oval_.related(point_symmetric_)

    assert (focus ^ oval_).hamming(unifocal_) > (focus ^ oval_).hamming(bifocal_)
    assert (focus ^ circle_).hamming(unifocal_) < (focus ^ circle_).hamming(bifocal_)

In [ ]:
P(circle, oval, point_symmetric, unifocal, bifocal)

In [ ]:
in_space = BHV.rand()

P(circle ^ in_space, oval ^ in_space, point_symmetric ^ in_space, unifocal ^ in_space, bifocal ^ in_space)

In [ ]:
assert not circle.related(circle ^ in_space)

## prototype based classification (MAJORITY acts a prototype)

In [ ]:
center = BHV.rand()
TL, TR, BL, BR = BHV.nrand(4)

In [ ]:
def encode(x, y):
    x_enc = BHV.random(x).select(TL, center) if x > 0 else BHV.random(-x).select(center, TR)
    y_enc = BHV.random(y).select(BL, center) if y > 0 else BHV.random(-y).select(center, BR)
    return BHV.majority([x_enc, y_enc])

In [ ]:
samples = 20
test = 20

In [ ]:
in_sphere = []
out_sphere = []
test_in_sphere = []
test_out_sphere = []
for i in range(samples + test):
    p = (random()*2 - 1, random()*2 - 1)
    r = hypot(*p) < 1
    if i >= samples:
        if r: test_in_sphere.append(p)
        else: test_out_sphere.append(p)
    else:
        if r: in_sphere.append(p)
        else: out_sphere.append(p)


pos = [encode(x, y) for x, y in in_sphere]
neg = [encode(x, y) for x, y in out_sphere]
test_pos = [encode(x, y) for x, y in test_in_sphere]
test_neg = [encode(x, y) for x, y in test_out_sphere]

sphere_prototype = BHV.majority(pos)
rest_prototype = BHV.majority(neg)


right = 0
test_right = 0
for i, p in enumerate(pos + neg + test_pos + test_neg):
    prediction = p.closest([rest_prototype, sphere_prototype])
    if i >= samples: test_right += prediction == (i < len(pos) + samples)
    else: right += prediction == (i < len(test_pos))

print(f"train acc {right/samples}")
print(f"test acc {test_right/test}")

train acc 0.75
test acc 0.45


## learning from examples (inductive reasoning, grandmother example)

In [ ]:
# relation utility
rel_subject = Perm.random()
rel_object = Perm.random()

# relations
mother_of = BHV.rand()
father_of = BHV.rand()
grandmother_of = BHV.rand()

In [ ]:
def apply_rel(rel, x, y):
  sx = rel_subject(rel) ^ x
  sy = rel_object(rel) ^ y
  return BHV.majority([sx, sy])

# our rule, read `xor` as "implied by" and `BHV.majority` as "and"
# note this is applied to multiple "datapoints" ...
def generate_sample():
  person_x = BHV.rand()
  person_y = BHV.rand()
  person_z = BHV.rand()

  mxy = apply_rel(mother_of, person_x, person_y)
  fyz = apply_rel(father_of, person_y, person_z)
  gxz = apply_rel(grandmother_of, person_x, person_z)

  return gxz ^ BHV.majority([mxy, fyz])

In [ ]:
datapoints = [generate_sample() for _ in range(15)]
grandmother_rule = BHV.majority(datapoints)
# ... and averaged out for higher accuracy

In [ ]:
# applying grandmother rule"
anna = BHV.rand()
bill = BHV.rand()
cid = BHV.rand()
anna_mother_of_bill = apply_rel(mother_of, anna, bill)
bill_father_of_cid = apply_rel(father_of, bill, cid)

actual_anna_grandmother_of_cid = apply_rel(grandmother_of, anna, cid)

calculated_anna_grandmother_of_cid = grandmother_rule ^ BHV.majority([anna_mother_of_bill, bill_father_of_cid])

assert calculated_anna_grandmother_of_cid.related(actual_anna_grandmother_of_cid, stdvs=4)

## checking if objects are related in a certain way (asymmetric biases, not just a space)

In [ ]:
def bias_ps(s, ps, o, frac=.2):
    mps = BHV.majority(ps)
    s_ = s.select_random(mps ^ o, 1. - frac)
    o_ = o.select_random(mps ^ s, 1. - frac)
    return s_, o_

def measure_bias(x, rel, y):
    rx = x.std_apart(rel)
    ry = y.std_apart(rel)
    return rx - ry

In [ ]:
# property graph
a, b, c, p, q, r = BHV.nrand(6)

In [ ]:
apq, cpq = bias_ps(a, [p, q], c)

assert abs(apq.zscore()) < 4 and abs(apq.zscore()) < 4

print("normal distance", apq.std_apart(cpq, relative=True))
print("bias", measure_bias(apq, c ^ BHV.majority([p, q]), cpq))
print("reverse bias", measure_bias(cpq, c ^ BHV.majority([p, q]), apq))

print("different relation", measure_bias(cpq, c ^ BHV.majority([r]), apq))

normal distance 0.7071067811865475
bias -8.043339635996972
reverse bias 8.352698852766082
different relation -0.04419417382416668


## deduction logic (MAJ and XOR with negation)

In [ ]:
raining, street_is_wet = BHV.nrand(2)

In [ ]:
# If it's raining, then the street is wet.
# It's raining.

# Therefore, the street is wet.

In [ ]:
implication = raining ^ ~street_is_wet
truth = raining
conclusion = truth ^ implication

assert not raining.related(conclusion)
assert street_is_wet.related(conclusion)

In [ ]:
implication = BHV.majority([~raining, street_is_wet])
truth = BHV.majority([raining, BHV.rand()])
conclusion = BHV.majority([implication, truth])

assert not raining.related(conclusion)
assert street_is_wet.related(conclusion)